In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('finishedResponses.csv')

/var/folders/3y/2jj6w99n11g766vg19kpr73h0000gn/T/ipykernel_18397/3381019248.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('finishedResponses.csv')


In [3]:
df.head()

,Unnamed: 0,localid,pos,ans,ttr,queryType,numTries,clipNum,questNum,subType,clipName,sessionNums,respEmo,respLevel,dispEmo,dispVal,dispLevel
0,13,SSI_1090000584,13,N_58,14516,1,0,768,1,4,1010_MTI_SAD_XX,1,N,58,S,50.0,X
1,17,SSI_1090000584,17,H_39,6977,1,0,2981,2,4,1037_IWL_HAP_XX,1,H,39,H,50.0,X
2,21,SSI_1090000584,21,S_79,5760,1,0,1736,3,4,1022_IWW_FEA_XX,1,S,79,F,50.0,X
3,25,SSI_1090000584,25,F_40,10512,1,0,5471,4,4,1067_WSI_HAP_XX,1,F,40,H,50.0,X
4,29,SSI_1090000584,29,N_88,6911,1,0,21,5,4,1001_TIE_FEA_XX,1,N,88,F,50.0,X


In [4]:
#Only save the queryType == 1 (Audio only) to align the setting
df = df[df['queryType'] == 1]
#Check
df['queryType'].unique()

array([1])

In [5]:
print(df['respEmo'].value_counts())

N    32145
A    10376
D     9588
F     8831
S     7404
H     4910
Name: respEmo, dtype: int64


In [6]:
#Clean the respLevel column as it includes some noises (unrelated alphabets, and numbers in string form)
#Convert the numeric string to numeric and the rest to NaN
df['respLevel'] =pd.to_numeric(df['respLevel'], errors='coerce')
#Drop rows with NaN in respLevel
df.dropna(subset=['respLevel'], inplace=True)
#Make sure the respLevel is integer
df['respLevel'] = df['respLevel'].astype(int)
#Check
df['respLevel'].unique()

array([ 58,  39,  79,  40,  88,  52,  36,  50,  68, 100,  63,  48,  37,
        55,  69,  60,  42,  65,  61,  51,  99,  80,  35,  66,  72,  26,
        24,  74,  47,  23,  89,  75,  87,  82,  38,  71,  78,  92,  97,
        15,  18,  13,  95,  43,  53,  67,  31,  91,  14,   5,   4,  17,
         9,  81,  34,  27,   7,   8,  12,  45,  32,  46,  22,  10,  77,
        30,  16,  20,  56,  29,  41,  25,  57,  44,  49,  76,  84,  19,
        28,   6,  64,  70,  85,  83,  90,  73,  33,  96,  93,  98,  62,
        86,  59,  54,  94,   1,  11,  21,   3,   2])

In [7]:
for count in df['respEmo'].value_counts().items():
    print(count)

('N', 32145)
('A', 10375)
('D', 9587)
('F', 8831)
('S', 7404)
('H', 4909)


In [8]:
from tqdm import tqdm
def dataFrameMaker(df):
    newDataFrame = pd.DataFrame(columns=['clipName', 
                                         'angry', 
                                         'disgust', 
                                         'fear', 
                                         'happy',
                                         'neutral', 
                                         'sad', 
                                         'low', 
                                         'medium', 
                                         'high'])
    
    for clipName in tqdm(df['clipName'].unique()):
        # Create a new dataframe per clip
        dfClip = df[df['clipName'] == clipName]
        totalNum = len(dfClip)
        content = {'clipName': clipName}
        # Get the emotion counts
        for emo in dfClip['respEmo'].value_counts().items():
            # From value_counts, get the emotion flag and voting counts
            # Devide the voting counts by the total number of votes to make it a probability distribution
            content[emo[0]] = emo[1] / totalNum
        
        # Set the placeholder for the emotional level counts
        low = 0
        medium = 0
        high = 0
        
        # Get the emotional level counts.
        # The emotional level is reported as an integer betweeen 0 - 100
        # The author of the dataset specified following threshold: 20: low, 50: medium, 80: high
        # Following the threshold, we can count the number of votes for each level
        for level in dfClip['respLevel']:
            if level < 20:
                low += 1
            elif level < 50:
                medium += 1
            else:
                high += 1
                
        # Normalize the counts to make it a probability distribution
        low /= totalNum
        medium /= totalNum
        high /= totalNum
                
        content['low'] = low
        content['medium'] = medium
        content['high'] = high
        
        # Append the content to the new dataframe
        newDataFrame = pd.concat([newDataFrame,dfClip], ignore_index=True)
        
    return newDataFrame
        

In [9]:
newDF = dataFrameMaker(df)
newDF.head()

newDF.to_csv('tabulatedData.csv', index=False)

100%|██████████| 7442/7442 [17:31<00:00,  7.08it/s]
